In [21]:
HTML(read(open("style.css"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\";\r\n    font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

# Play a Chess Game

In [22]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random
# Pkg.add("PyPlot")
using PyPlot
# Pkg.add("NBInclude")
using NBInclude

In [23]:
function printGame(game)
#     typeof(game)
    display(board(game))
end

printGame (generic function with 1 method)

In [24]:
game = SimpleGame()
board(game)
printGame(game)

Board (rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  p  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  P  P  P  P 
 R  N  B  Q  K  B  N  R

In [25]:
@nbinclude("Random Chess.ipynb")

playMove (generic function with 1 method)

In [26]:
@nbinclude("Chess Games Save.ipynb")

saveGameToPGN (generic function with 1 method)

In [27]:
@nbinclude("ValidateUserInput.ipynb")

is_legal_move (generic function with 1 method)

In [28]:
@nbinclude("EvaluatePosition.ipynb")

keine Figur vorhandenSubCapture

100000

In [29]:
@nbinclude("Minimax.ipynb")

keine Figur vorhandenSubCapture440
Move[Move(a8a7)](100003, Move(a8a7))  0.253640 seconds (307.35 k allocations: 19.136 MiB, 3.01% gc time, 58.09% compilation time)
-680
(185, Move(c3a1))  0.361562 seconds (2.54 M allocations: 98.915 MiB, 5.00% gc time, 32.81% compilation time)
-680
                        
             k          
                        
             p          
                        
       b  k             
                        
 q                      
8/4k3/8/4p3/8/2BK4/8/q7 w - -
-----------------------
Move(c3a5)
                        
             k          
                        
 b           p          
                        
          k             
                        
 q                      
8/4k3/8/B3p3/8/3K4/8/q7 b - -
-1015

-----------------------
Move(c3a1)
                        
             k          
                        
             p          
                        
          k             
                        
 b  

 q                      
8/4k3/8/4p3/1B6/3K4/8/q7 b - -
-670

-----------------------
Move(c3b2)
                        
             k          
                        
             p          
                        
          k             
    b                   
 q                      
8/4k3/8/4p3/8/3K4/1B6/q7 b - -
-1040

-----------------------
Move(c3d4)
                        
             k          
                        
             p          
          b             
          k             
                        
 q                      
8/4k3/8/4p3/3B4/3K4/8/q7 b - -
-1025

-----------------------
Move(c3d2)
                        
             k          
                        
             p          
                        
          k             
          b             
 q                      
8/4k3/8/4p3/8/3K4/3B4/q7 b - -
-705

-----------------------
Move(c3e5)
                        
             k          
                        
          

          k             
                        
 q           b          
8/4k3/8/4p3/8/3K4/8/q3B3 b - -
-1015

-----------------------
Move(d3c4)
                        
             k          
                        
             p          
       k                
       b                
                        
 q                      
8/4k3/8/4p3/2K5/2B5/8/q7 b - -
-695

-----------------------
Move(d3c2)
                        
             k          
                        
             p          
                        
       b                
       k                
 q                      
8/4k3/8/4p3/8/2B5/2K5/q7 b - -
-660

-----------------------
Move(d3d2)
                        
             k          
                        
             p          
                        
       b                
          k             
 q                      
8/4k3/8/4p3/8/2B5/3K4/q7 b - -
-660

-----------------------
Move(d3e4)
                        
           

                        
             p          
             k          
       b                
                        
 q                      
8/4k3/8/4p3/4K3/2B5/8/q7 b - -
-1065

-----------------------
Move(d3e3)
                        
             k          
                        
             p          
                        
       b     k          
                        
 q                      
8/4k3/8/4p3/8/2B1K3/8/q7 b - -
-1025

-----------------------
Move(d3e2)
                        
             k          
                        
             p          
                        
       b                
             k          
 q                      
8/4k3/8/4p3/8/2B5/4K3/q7 b - -
-1015

(185, Move(c3a1))  1.720052 seconds (424.28 k allocations: 18.791 MiB, 3.95% compilation time)


`printGame` function prints the board to standard output. Settings can be set here. 

`printWin(game)` prints the result of the game as a text message.

In [30]:
function printWin(board)
    if ischeckmate(board)
        color = sidetomove(board)
        color == "WHITE" ? color = "BLACK" : color = "WHITE"
        print(color * " wins by checkmate" )
    elseif isstalemate(board)
        print("Stalemate")
    elseif ismaterialdraw(board)
        print("Draw by insufficient material")
    elseif isrule50draw(board)
        print("Draw by 50 move draw")
    end
end

printWin (generic function with 1 method)

In [31]:
function playMove(game)
    eval = playMinimaxMove(game, 3)
    print(eval)
    return eval
end

playMove (generic function with 1 method)

In [32]:
function playGame(color)
    # Random.seed!(seed)
    # validation
    game = SimpleGame()
    printGame(game)
    setGameHeaders(game)
    
    current_boardscore = 0  # evaluate_position(startboard())
    
    println("what's your name?")
    username = readline()
    
    # Offset move if player is black
    if color != "black" && color != "white"
        print("Invalid color. Choose white or black")   
        return
    elseif color == "black"
        setheadervalue!(game, "Black", username)
        current_boardscore = playMove(game, current_boardscore)
        printGame(game)
    elseif color == "white"
        setheadervalue!(game, "White", username)
    end

    while true
        # User Move
        if !isterminal(game)
            println("Make a move")
            userInput = readline()
            userMove = movefromstring(userInput)
            if(userMove == nothing)
                println("Invalid User Input. Please use the format [source] [destination]. Example(e2e4)")
                continue
            end
            if(!is_legal_move(game, userMove))
                println("Invalid Chess move. Please make a other move.")
                continue
            else
                current_boardscore = evaluate_move(board(game), userMove, current_boardscore)
                domove!(game, userMove)
            end
            printGame(game)
        else
            break
        end
        
        # Delay
        sleep(1)
        
        # Engine Move
        if !isterminal(game)
            current_boardscore = playMove(game)
            printGame(game)
        else
            break
        end
    end
    saveGameToPGN(game)
    printWin(board(game))
    return game
end

playGame (generic function with 1 method)

In [33]:
playGame("white")

Board (rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  p  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  P  P  P  P 
 R  N  B  Q  K  B  N  R

what's your name?
stdin> e2e4
Make a move
stdin> e2e4


Board (rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq -):
 r  n  b  q  k  b  n  r 
 p  p  p  p  p  p  p  p 
 -  -  -  -  -  -  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  P  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

-55

Board (rnbqkb1r/pppppppp/5n2/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq -):
 r  n  b  q  k  b  -  r 
 p  p  p  p  p  p  p  p 
 -  -  -  -  -  n  -  - 
 -  -  -  -  -  -  -  - 
 -  -  -  -  P  -  -  - 
 -  -  -  -  -  -  -  - 
 P  P  P  P  -  P  P  P 
 R  N  B  Q  K  B  N  R

...

Board (3r2k1/pp2qp2/3p3Q/3B1P2/4P1P1/1P1R1K2/2r3P1/7R w - -):
 -  -  -  r  -  -  k  - 
 p  p  -  -  q  p  -  - 
 -  -  -  p  -  -  -  Q 
 -  -  -  B  -  P  -  - 
 -  -  -  -  P  -  P  - 
 -  P  -  R  -  K  -  - 
 -  -  r  -  -  -  P  - 
 -  -  -  -  -  -  -  R

Make a move
stdin> h6h8


Board (3r2kQ/pp2qp2/3p4/3B1P2/4P1P1/1P1R1K2/2r3P1/7R b - -):
 -  -  -  r  -  -  k  Q 
 p  p  -  -  q  p  -  - 
 -  -  -  p  -  -  -  - 
 -  -  -  B  -  P  -  - 
 -  -  -  -  P  -  P  - 
 -  P  -  R  -  K  -  - 
 -  -  r  -  -  -  P  - 
 -  -  -  -  -  -  -  R

[Event "Random Chess"]
[Site "DHBW Mannheim"]
[Date "2023.01.09"]
[Round "1"]
[White "e2e4"]
[Black "Random Player"]
[Result "*"]

1. e4 Nf6 2. Nc3 Nc6 3. Nf3 e5 4. d4 exd4 5. Nxd4 Nxd4 6. Qxd4 Bd6 7. Bc4 O-O 8. O-O Bb4 9. Nd5 Nxd5 10. Bxd5 Qe7 11. b3 Qd6 12. Bb2 Bc3 13. Bxc3 Qg6 14. Rad1 d6 15. f4 Bg4 16. Rd2 c5 17. Qe3 Rab8 18. Qg3 Rfd8 19. h3 Qh5 20. hxg4 Qg6 21. f5 Qg5 22. Kf2 c4 23. Rh1 cxb3 24. axb3 Rbc8 25. Rd3 Rc5 26. Ke2 Qe7 27. Qh2 h6 28. Bxg7 Rxc2+ 29. Kf3 Kxg7 30. Qxh6+ Kg8 31. Qh8# *

WHITE wins by checkmate

SimpleGame (e2e4 vs Random Player, DHBW Mannheim 2023):
 1. e4 Nf6 2. Nc3 Nc6 3. Nf3 e5 4. d4 exd4 5. Nxd4 Nxd4 6. Qxd4 Bd6 7. Bc4 O-O 8. O-O Bb4 9. Nd5 Nxd5 10. Bxd5 Qe7 11. b3 Qd6 12. Bb2 Bc3 13. Bxc3 Qg6 14. Rad1 d6 15. f4 Bg4 16. Rd2 c5 17. Qe3 Rab8 18. Qg3 Rfd8 19. h3 Qh5 20. hxg4 Qg6 21. f5 Qg5 22. Kf2 c4 23. Rh1 cxb3 24. axb3 Rbc8 25. Rd3 Rc5 26. Ke2 Qe7 27. Qh2 h6 28. Bxg7 Rxc2+ 29. Kf3 Kxg7 30. Qxh6+ Kg8 31. Qh8# *